## For Second project of Strawberry image labeling
> **Idea: get information about number of object in each images using YOLOv8 medium model(trained)** <br>
> 이전 500장 라벨링 데이터를 통해 훈련시킨 YOLOv8모델을 통해 이미지에 어떤 객체가 얼마나 있는지 대략적인 정보를 얻고 <br>
> 정보를 바탕으로 이미지 1500장 선별, 1500장 -> 1000장 수렴 계획


In [1]:
from ultralytics import YOLO
from PIL import Image
import cv2

#n= 1000 
File_path=[]
File_names=[]
label_names = ['Flower','Before_blooming','Receptacle','Green_small_fruit','Strawberry_1','Strawberry_2','Strawberry_3']
N_flower=[]
N_fold_flower=[]
N_receptacle=[]
N_greenSF=[]
N_str1=[]
N_str2=[]
N_str3=[]

model = YOLO("/home/hyeonji/YOLOv8/agtech_lab/Y8_medium_Result/weights/Y8_str_medium_best.pt")

In [2]:
def add_list():
    N_flower.append(0)
    N_fold_flower.append(0)
    N_receptacle.append(0)
    N_greenSF.append(0)
    N_str1.append(0)
    N_str2.append(0)
    N_str3.append(0)
    
def box_class_counter(results, idx):
    box_class = -1       
    add_list()
    #print("----------------",idx)
    
    for r in results.boxes:
                
        box_class = int(r.cls.tolist()[0])
        
        if box_class == 0:
            N_flower[idx] +=1

        elif box_class == 1:
            N_fold_flower[idx] +=1

        elif box_class == 2:
            N_receptacle[idx] +=1

        elif box_class == 3:
            N_greenSF[idx] +=1

        elif box_class == 4:
            N_str1[idx] +=1

        elif box_class == 5:
            N_str2[idx] +=1

        elif box_class == 6:
            N_str3[idx] +=1
        else:
            print(box_class)
   

In [3]:
import pandas as pd
import os
path = "/home/hyeonji/New_storage/alarad_img_P2_1000/V2/"

Date = []  # 촬영일
Time = []  # 촬영시
Line = []  # 촬영 라인
View = []  # Topview, sideview
Position = []  # 촬영위치
filename = []  # 파일명
Rootdirs = []  # 추후 파일 찾기 편하려고 만든 루트
D_N = []  # 오전 오후 구분

for root, subdirs, files in os.walk(path):
    for i in files:
            if len(i.split('_')) > 1:
                try:
                    Date.append(i.split('_')[0])
                    Time.append(i.split('_')[1])
                    if int(i.split('_')[1]) > 160000:
                        D_N.append(1)
                    else:
                        D_N.append(0)  # 낮
                    Line.append(i.split('_')[2])
                    View.append(i.split('_')[3])
                    Position.append(i.split('_')[4])
                    filename.append(i)
                    Rootdirs.append(root)

                except:
                    print(i)
#날짜 시간순 정렬 필 (그러면 원래 디렉토리내 파일 순서오 ㅏ같아짐)
myData = {
    'filename': filename, 'Date': Date, 'Time': Time, 'Line': Line, 'View': View, 'Position': Position, 'Rootdirs': Rootdirs, 'isDay': D_N
}
myDF = pd.DataFrame(myData)
myDF = myDF.sort_values(by=['Date','Time'],ignore_index=True)
myDF

,filename,Date,Time,Line,View,Position,Rootdirs,isDay
0,221217_082642_L1_V2_P11_D0410.png,221217,082642,L1,V2,P11,/home/hyeonji/New_storage/alarad_img_P2_1000/V2/,0
1,221217_082954_L1_V2_P18_D0690.png,221217,082954,L1,V2,P18,/home/hyeonji/New_storage/alarad_img_P2_1000/V2/,0
2,221217_162758_L1_V2_P17_D0650.png,221217,162758,L1,V2,P17,/home/hyeonji/New_storage/alarad_img_P2_1000/V2/,1
3,221217_163138_L1_V2_P25_D0970.png,221217,163138,L1,V2,P25,/home/hyeonji/New_storage/alarad_img_P2_1000/V2/,1
4,221217_163301_L1_V2_P28_D1090.png,221217,163301,L1,V2,P28,/home/hyeonji/New_storage/alarad_img_P2_1000/V2/,1
...,...,...,...,...,...,...,...,...
995,230312_162438_L1_V2_P10_D0370.png,230312,162438,L1,V2,P10,/home/hyeonji/New_storage/alarad_img_P2_1000/V2/,1
996,230312_162535_L1_V2_P12_D0450.png,230312,162535,L1,V2,P12,/home/hyeonji/New_storage/alarad_img_P2_1000/V2/,1
997,230312_164227_L4_V2_P10_D0370.png,230312,164227,L4,V2,P10,/home/hyeonji/New_storage/alarad_img_P2_1000/V2/,1
998,230312_164934_L4_V2_P23_D0890.png,230312,164934,L4,V2,P23,/home/hyeonji/New_storage/alarad_img_P2_1000/V2/,1


In [5]:
results = model.predict(source='/home/hyeonji/New_storage/alarad_img_P2_1000/V2_3496_3496/', conf=0.1, show=False)


Ultralytics YOLOv8.0.8 🚀 Python-3.8.16 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3080, 10008MiB)
Fusing layers... 
Model summary: 218 layers, 25843813 parameters, 0 gradients, 78.7 GFLOPs


In [6]:
idx = 0
for result in results:
    #하나의 이미지 당,, result 
    box_class_counter(result,idx)
    idx += 1

In [7]:
import openpyxl

#결과 엑셀화
img_result = {'filename': myDF['filename'], 'Date': myDF['Date'], 'Time': myDF['Time'], 'Line': myDF['Line'], 'D_N': myDF['isDay'],'flowers': N_flower, 'before_blooming': N_fold_flower, 'receptacle':N_receptacle, 'green_small_fruit':N_greenSF, 'strawberry1':N_str1, 'strawberry2':N_str2, 'strawberry3':N_str3 }

ir_DF=pd.DataFrame(img_result)
ir_DF.to_excel('/home/hyeonji/YOLOv8/agtech_lab/Crop_img_select_support_result.xlsx')

In [8]:
ir_DF

,filename,Date,Time,Line,D_N,...,receptacle,green_small_fruit,strawberry1,strawberry2,strawberry3
0,221217_082642_L1_V2_P11_D0410.png,221217,082642,L1,0,...,13,8,6,0,0
1,221217_082954_L1_V2_P18_D0690.png,221217,082954,L1,0,...,6,7,3,0,0
2,221217_162758_L1_V2_P17_D0650.png,221217,162758,L1,1,...,4,10,3,0,0
3,221217_163138_L1_V2_P25_D0970.png,221217,163138,L1,1,...,4,6,1,0,0
4,221217_163301_L1_V2_P28_D1090.png,221217,163301,L1,1,...,6,11,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...
995,230312_162438_L1_V2_P10_D0370.png,230312,162438,L1,1,...,13,12,15,4,8
996,230312_162535_L1_V2_P12_D0450.png,230312,162535,L1,1,...,4,25,11,10,5
997,230312_164227_L4_V2_P10_D0370.png,230312,164227,L4,1,...,1,34,16,1,1
998,230312_164934_L4_V2_P23_D0890.png,230312,164934,L4,1,...,2,21,7,11,2


In [ ]:
import pandas as pd
import os
path = "/home/hyeonji/New_storage/alarad_img_P2_1000/V1/"

Date = []  # 촬영일
Time = []  # 촬영시
Line = []  # 촬영 라인
View = []  # Topview, sideview
Position = []  # 촬영위치
filename = []  # 파일명
Rootdirs = []  # 추후 파일 찾기 편하려고 만든 루트
D_N = []  # 오전 오후 구분

for root, subdirs, files in os.walk(path):
    for i in files:
            if len(i.split('_')) > 1:
                try:
                    Date.append(i.split('_')[0])
                    Time.append(i.split('_')[1])
                    if int(i.split('_')[1]) > 160000:
                        D_N.append(1)
                    else:
                        D_N.append(0)  # 낮
                    Line.append(i.split('_')[2])
                    View.append(i.split('_')[3])
                    Position.append(i.split('_')[4])
                    filename.append(i)
                    Rootdirs.append(root)

                except:
                    print(i)
#날짜 시간순 정렬 필 (그러면 원래 디렉토리내 파일 순서오 ㅏ같아짐)
myData = {
    'filename': filename, 'Date': Date, 'Time': Time, 'Line': Line, 'View': View, 'Position': Position, 'Rootdirs': Rootdirs, 'isDay': D_N
}
myDF = pd.DataFrame(myData)
myDF = myDF.sort_values(by=['Date','Time'],ignore_index=True)
myDF.to_excel('/home/hyeonji/YOLOv8/agtech_lab/img_select_support_resultV1.xlsx')